In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from IPython.core.pylabtools import figsize
import matplotlib.pyplot as plt
import os.path
import pandas as pd
import collections
import seaborn as sns

In [2]:
lmap = pd.read_csv(os.path.join('linkage_map','LEPmap','with_paralogs','final','PS_chum_map_2015.txt'), sep = '\t')
lmap = lmap.drop(['cM_OLD', 'paper1_LG'], axis = 1)
lmap= lmap.drop_duplicates('LEPname')
lmap = lmap.sort_values(['LEP_LG', 'cM'])
lmap.tail()

,contig,resolved_locus,stacks_CatID,stacks_SNP,LEPname,LEP_LG,cM
6988,c2953,2953_C,16919,16919_25,2902,37,56.28
6989,c56816,56816_C,39923,39923_44,7366,37,56.28
6990,c47181,47181_C,31760,31760_82,5379,37,56.28
6991,c7090,7090_A,51610,51610_40,10169,37,56.28
6992,c56688,56688_C,39817,39817_67,7337,37,66.16


In [3]:
lmap['fakeBP'] = (lmap['cM']*1000000 + lmap.index).astype(np.int)

In [4]:
lmap.tail()

,contig,resolved_locus,stacks_CatID,stacks_SNP,LEPname,LEP_LG,cM,fakeBP
6988,c2953,2953_C,16919,16919_25,2902,37,56.28,56286988
6989,c56816,56816_C,39923,39923_44,7366,37,56.28,56286989
6990,c47181,47181_C,31760,31760_82,5379,37,56.28,56286990
6991,c7090,7090_A,51610,51610_40,10169,37,56.28,56286991
6992,c56688,56688_C,39817,39817_67,7337,37,66.16,66166992


In [5]:
mapfile = lmap[['LEP_LG', 'LEPname', 'cM', 'fakeBP']]
mapfile.to_csv('/home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci.map', 
               sep = '\t', index = False, header = False)

In [6]:
shapeit_mapfile = lmap[['LEP_LG','fakeBP', 'cM']].copy()
shapeit_mapfile['rrate'] = 1.0
shapeit_mapfile = shapeit_mapfile[['LEP_LG', 'fakeBP', 'rrate', 'cM']].copy()
shapeit_mapfile.head()

,LEP_LG,fakeBP,rrate,cM
0,1,0,1,0
2,1,2,1,0
6,1,6,1,0
12,1,12,1,0
14,1,14,1,0


In [7]:
gb = shapeit_mapfile.groupby('LEP_LG')

In [8]:
for xx in range(1, 38):
    to_write = gb.get_group(xx).drop('LEP_LG', axis = 1)
    to_write.to_csv('linkage_map/centromere_placement/shapeit/collapsed_loci.{}.shapeitmap'.format(xx), sep = ' ', header = True, index = False)

In [9]:
lepmap_infile = pd.read_csv(os.path.join('linkage_map','LEPmap','with_paralogs', 'collapsed_loci.lepmap'), sep = '\t')
lepmap_infile.head()

,#family,name,sire,dam,sex,blank,10001,10003_A,1000_C,10012,...,Oke_ndub3-58,Oke_pnrc2-78,Oke_psmd9-57,Oke_rab5a-117,Oke_sylc-90,Oke_thic-84,Oke_txnrd1-74,Oke_u0602-244,Oke_u217-172,Oke_zn593-152
0,fam_08,fam_08_Dam,0,0,2,0,1 1,1 1,1 1,1 1,...,1 1,1 1,1 1,1 1,1 1,1 1,1 1,1 1,1 1,1 1
1,fam_08,fam_08_Sire,0,0,1,0,1 2,1 2,1 2,1 2,...,1 2,1 2,1 2,1 2,1 2,1 2,1 2,1 2,1 2,1 2
2,fam_08,CMUW10X08H_0002,fam_08_Sire,fam_08_Dam,0,0,1 1,1 2,0 0,0 0,...,0 0,1 2,0 0,0 0,1 1,1 2,1 2,1 1,1 1,1 2
3,fam_08,CMUW10X08H_0003,fam_08_Sire,fam_08_Dam,0,0,1 2,1 1,0 0,0 0,...,0 0,1 2,0 0,0 0,1 1,1 1,1 2,1 2,1 1,1 1
4,fam_08,CMUW10X08H_0005,fam_08_Sire,fam_08_Dam,0,0,1 2,1 2,0 0,0 0,...,0 0,1 2,0 0,0 0,1 1,1 2,1 2,1 2,1 2,1 2


In [10]:
lepmap_infile.columns = lepmap_infile.columns[:6].tolist() + list(range(1,len(lepmap_infile.columns[6:])+1))

## subset and reorder base on linkage map

In [17]:
ped = lepmap_infile[lepmap_infile.columns[:6].tolist() + lmap['LEPname'].tolist()]
ped.head()

,#family,name,sire,dam,sex,blank,4584,7377,6722,1923,...,4552,6213,2558,5198,5942,2902,7366,5379,10169,7337
0,fam_08,fam_08_Dam,0,0,2,0,1 1,1 1,1 1,1 1,...,1 1,1 1,1 1,1 1,1 1,1 1,1 1,1 1,1 1,1 1
1,fam_08,fam_08_Sire,0,0,1,0,1 2,1 2,1 2,1 2,...,1 2,1 2,1 2,1 2,1 2,1 2,1 2,1 2,1 2,1 2
2,fam_08,CMUW10X08H_0002,fam_08_Sire,fam_08_Dam,0,0,1 1,1 1,1 1,0 0,...,1 1,1 2,0 0,1 1,1 2,0 0,0 0,0 0,1 1,0 0
3,fam_08,CMUW10X08H_0003,fam_08_Sire,fam_08_Dam,0,0,1 1,1 1,1 1,0 0,...,1 2,1 1,0 0,0 0,1 1,0 0,0 0,0 0,1 2,0 0
4,fam_08,CMUW10X08H_0005,fam_08_Sire,fam_08_Dam,0,0,1 1,1 1,1 1,0 0,...,1 1,1 2,0 0,1 1,1 2,0 0,0 0,0 0,1 1,0 0


## drop the two small families

In [20]:
ped = ped[ped['#family']=='fam_08']

In [21]:
ped.to_csv('/home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci.ped', 
               sep = '\t', index = False, header = False)

## Split by chromosome in PLINK

In [22]:
for chrom in range(1, 38):
    !plink --file /home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci \
           --autosome-num 50 --chr {chrom} --recode --geno .5 \
           --out {'/home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci.chrom{}'.format(chrom)}

PLINK v1.90b3w 64-bit (3 Sep 2015)         https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci.chrom1.log.
Options in effect:
  --autosome-num 50
  --chr 1
  --file /home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci
  --geno .5
  --out /home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci.chrom1
  --recode

32127 MB RAM detected; reserving 16063 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (331 variants, 177 samples).
--file:
/home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci.chrom1-temporary.bed
+
/home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci.chrom1-temporary.b

## Run SHAPEIT2 to phase

In [ ]:
for chrom in range(1, 38):
    !'/home/ipseg/Programs/SHAPEIT/shapeit.v2.r837.GLIBCv2.12.Linux.static/bin/shapeit' \
    --input-ped {'/home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci.chrom{}.ped'.format(chrom)} \
      {'/home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci.chrom{}.map'.format(chrom)} \
    --duohmm \
    --states 100 --window 10 \
    --effective-size 1000 \
    --thread 7 \
    --burn 100 --prune 100 --main 500 \
    --input-map {'/home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci.{}.shapeitmap'.format(chrom)} \
    --output-max {'/home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci.chrom{}.phased'.format(chrom)} \
    --output-graph {'/home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci.chrom{}.phased.graph'.format(chrom)}




Segmented HAPlotype Estimation & Imputation Tool
  * Authors : Olivier Delaneau, Jared O'Connell, Jean-François Zagury, Jonathan Marchini
  * Contact : send an email to the OXSTATGEN mail list https://www.jiscmail.ac.uk/cgi-bin/webadmin?A0=OXSTATGEN
  * Webpage : https://mathgen.stats.ox.ac.uk/shapeit
  * Version : v2.r837
  * Date    : 30/10/2015 23:55:42
  * LOGfile : [shapeit_30102015_23h55m42s_e3ca546d-3d04-471d-b3c2-ff5e0794565e.log]

MODE -phase : PHASING GENOTYPE DATA
  * Autosome (chr1 ... chr22)
  * Window-based model (SHAPEIT v2)
  * MCMC iteration
  * duoHMM is used to refine family haplotypes

Parameters :
  * Seed : 1446274542
  * Parallelisation: 7 threads
  * Ref allele is NOT aligned on the reference genome
  * MCMC: 700 iterations [100 B + 1 runs of 100 P + 500 M]
  * Model: 100 states per window [100 H + 0 PM + 0 R + 0 COV ] / Windows of ~10.0 Mb / Ne = 1000

Reading site list in [/home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/co

## Load phase data 
because of how we code genotypes for LEPmap - we want the sire phase:
    - family 8- line 4 
    - family 1- line 181
    - family 9- line 217
    
4+2*(x-2) = (8,9) (362, 363), [434, 435]

    
this corresponds to columns:
    - [7,8]
    - [361, 362]
    - [433, 434]


In [34]:
aa = None
for chrom in range(1, 38):
    haplotypes = pd.read_csv('/home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci.chrom{}.phased.haps'.format(chrom), 
           sep = ' ', header = None)
    fam8 = haplotypes.iloc[:,[7,8]]
    #fam1 = haplotypes.iloc[:,[361, 362]]
    #fam9 = haplotypes.iloc[:,[433, 434]]

    parental_haplotypes = fam8
    #parental_haplotypes = pd.merge(pd.merge(fam8, fam1, right_index=True, left_index=True), fam9, right_index=True, left_index=True)
    
    phase8 = parental_haplotypes[7] > parental_haplotypes[8]
    #phase1 = parental_haplotypes[361] > parental_haplotypes[362]
    #phase9 = parental_haplotypes[433] > parental_haplotypes[434]
    
    LEPnames = pd.read_csv('/home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci.chrom{}.map'.format(chrom), 
                           sep = '\t', header = None)

    LEPnames = LEPnames[1].values.tolist()
    #print chrom, LEPnames[-1]
    current_phase = pd.DataFrame({'LEPname':LEPnames, 
                                  'fam_08':phase8
                                  #'fam_01':phase1, 
                                  #'fam_09':phase9
                                 } )
    if aa is None:
        aa = current_phase
    else:
        aa = aa.append(current_phase)
parental_phases = aa
    
    
    
    

In [35]:
parental_phases.to_csv('/home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/parental_phase.txt', 
                       sep = '\t', index = False)

In [ ]:
aa.tail(50)

In [ ]:
chr9_phased = pd.read_csv('/home/ipseg/Desktop/waples/chum_populations/linkage_map/centromere_placement/shapeit/collapsed_loci.chrom{}.map'.format(chrom), 
           sep = ' ', header = None)

fam8 = chr9_phased.iloc[:,[7,8]]
fam1 = chr9_phased.iloc[:,[361, 362]]
fam9 = chr9_phased.iloc[:,[433, 434]]

parental_haplotypes = pd.merge(pd.merge(fam8, fam1, right_index=True, left_index=True), fam9, right_index=True, left_index=True)
parental_haplotypes.head()

In [ ]:
phase8 = parental_haplotypes[7] > parental_haplotypes[8]
phase1 = parental_haplotypes[361] > parental_haplotypes[362]
phase9 = parental_haplotypes[433] > parental_haplotypes[434]

In [ ]:
lmap

# For each locus within each family:
if the phase is False:
switch the inferred phase of all offspring 

In [ ]:
phase9

In [ ]:
pd.merge(chr9_phased.iloc[:,[8,9]], chr9_phased.iloc[:,[362,363]], left_index=True, right_index=True)